In [1]:
import copy
import glob
import os
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from arguments import get_args

#from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
#from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
#from baselines.common.vec_env.vec_normalize import VecNormalize
from all_stuff import * # this has the above modules consolidated into a single file. god this was a bitch

from envs import make_env # had to manually add some files into directory for env to reference bc baselines 
# modules not working right

from kfac import KFACOptimizer
from model import CNNPolicy, MLPPolicy
from storage import RolloutStorage
from visualize import visdom_plot

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):


In [14]:
from visdom import Visdom

In [2]:
class args:
    def __init__(self):
        self.env_name='PongNoFrameskip-v4'
        self.seed=1
        self.log_dir=''
        self.save_dir='saved_models'
        self.cuda=True
        self.algo='a2c'
        self.num_stack=4
        self.num_steps=5
        self.num_processes=16
        self.recurrent_policy=False
        self.vis=False
        self.lr=7e-4
        self.eps=1e-5
        self.alpha=.99
        self.max_grad_norm=.5
        self.value_loss_coef=.5
        self.entropy_coef=.1
        self.num_frames=3e6
        self.use_gae=False
        self.gamma=.99
        self.tau=.95
        self.save_interval=1000
        self.log_interval=100
        self.vis_interval=100
        self.from_saved_model=True
        
args = args()

save_path = os.path.join(args.save_dir, args.algo)
SAVE_PATH = os.path.join(save_path, args.env_name + ".pt")

In [3]:
num_updates = int(args.num_frames) // args.num_steps // args.num_processes

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

try:
    os.makedirs(args.log_dir)
except OSError:
    files = glob.glob(os.path.join(args.log_dir, '*.monitor.csv'))
    for f in files:
        os.remove(f)

In [5]:
def main():
    print("#######")
    print("WARNING: All rewards are clipped or normalized so you need to use a monitor (see envs.py) or visdom plot to get true rewards")
    print("#######")

    os.environ['OMP_NUM_THREADS'] = '1'

    if args.vis:
        from visdom import Visdom
        viz = Visdom()
        win = None

    envs = [make_env(args.env_name, args.seed, i, args.log_dir)
                for i in range(args.num_processes)]

    if args.num_processes > 1:
        envs = SubprocVecEnv(envs)
    else:
        envs = DummyVecEnv(envs)

    if len(envs.observation_space.shape) == 1:
        envs = VecNormalize(envs)

    obs_shape = envs.observation_space.shape
    obs_shape = (obs_shape[0] * args.num_stack, *obs_shape[1:])

    if args.from_saved_model:
        print("loading saved model from ", SAVE_PATH)
        actor_critic = torch.load(SAVE_PATH)
    else:
        if len(envs.observation_space.shape) == 3:
            actor_critic = CNNPolicy(obs_shape[0], envs.action_space, args.recurrent_policy)
        else:
            assert not args.recurrent_policy, \
                "Recurrent policy is not implemented for the MLP controller"
            actor_critic = MLPPolicy(obs_shape[0], envs.action_space)

    if envs.action_space.__class__.__name__ == "Discrete":
        action_shape = 1
    else:
        action_shape = envs.action_space.shape[0]

    if args.cuda:
        actor_critic.cuda()

    if args.algo == 'a2c':
        optimizer = optim.RMSprop(actor_critic.parameters(), args.lr, eps=args.eps, alpha=args.alpha)
    elif args.algo == 'ppo':
        optimizer = optim.Adam(actor_critic.parameters(), args.lr, eps=args.eps)
    elif args.algo == 'acktr':
        optimizer = KFACOptimizer(actor_critic)

    rollouts = RolloutStorage(args.num_steps, args.num_processes, obs_shape, envs.action_space,\
                              actor_critic.state_size)
    current_obs = torch.zeros(args.num_processes, *obs_shape)

    def update_current_obs(obs):
        shape_dim0 = envs.observation_space.shape[0]
        obs = torch.from_numpy(obs).float()
        if args.num_stack > 1:
            current_obs[:, :-shape_dim0] = current_obs[:, shape_dim0:]
        current_obs[:, -shape_dim0:] = obs

    obs = envs.reset()
    update_current_obs(obs)

    rollouts.observations[0].copy_(current_obs)

    # These variables are used to compute average rewards for all processes.
    episode_rewards = torch.zeros([args.num_processes, 1])
    final_rewards = torch.zeros([args.num_processes, 1])

    if args.cuda:
        current_obs = current_obs.cuda()
        rollouts.cuda()

    start = time.time()
    for j in range(num_updates):
        for step in range(args.num_steps):
            # Sample actions
            value, action, action_log_prob, states = actor_critic.act(Variable(rollouts.observations[step], volatile=True),
                                                                      Variable(rollouts.states[step], volatile=True),
                                                                      Variable(rollouts.masks[step], volatile=True))
            cpu_actions = action.data.squeeze(1).cpu().numpy()

            # Obser reward and next obs
            obs, reward, done, info = envs.step(cpu_actions)
            reward = torch.from_numpy(np.expand_dims(np.stack(reward), 1)).float()
            episode_rewards += reward

            # If done then clean the history of observations.
            masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
            final_rewards *= masks
            final_rewards += (1 - masks) * episode_rewards
            episode_rewards *= masks

            if args.cuda:
                masks = masks.cuda()

            if current_obs.dim() == 4:
                current_obs *= masks.unsqueeze(2).unsqueeze(2)
            else:
                current_obs *= masks

            update_current_obs(obs)
            rollouts.insert(step, current_obs, states.data, action.data, action_log_prob.data, value.data, reward, masks)

        next_value = actor_critic(Variable(rollouts.observations[-1], volatile=True),
                                  Variable(rollouts.states[-1], volatile=True),
                                  Variable(rollouts.masks[-1], volatile=True))[0].data

        rollouts.compute_returns(next_value, args.use_gae, args.gamma, args.tau)

        if args.algo in ['a2c', 'acktr']:
            values, action_log_probs, dist_entropy, states = actor_critic.evaluate_actions(Variable(rollouts.observations[:-1].view(-1, *obs_shape)),
                                                                                           Variable(rollouts.states[0].view(-1, actor_critic.state_size)),
                                                                                           Variable(rollouts.masks[:-1].view(-1, 1)),
                                                                                           Variable(rollouts.actions.view(-1, action_shape)))

            values = values.view(args.num_steps, args.num_processes, 1)
            action_log_probs = action_log_probs.view(args.num_steps, args.num_processes, 1)

            advantages = Variable(rollouts.returns[:-1]) - values
            value_loss = advantages.pow(2).mean()

            action_loss = -(Variable(advantages.data) * action_log_probs).mean()

            if args.algo == 'acktr' and optimizer.steps % optimizer.Ts == 0:
                # Sampled fisher, see Martens 2014
                actor_critic.zero_grad()
                pg_fisher_loss = -action_log_probs.mean()

                value_noise = Variable(torch.randn(values.size()))
                if args.cuda:
                    value_noise = value_noise.cuda()

                sample_values = values + value_noise
                vf_fisher_loss = -(values - Variable(sample_values.data)).pow(2).mean()

                fisher_loss = pg_fisher_loss + vf_fisher_loss
                optimizer.acc_stats = True
                fisher_loss.backward(retain_graph=True)
                optimizer.acc_stats = False

            optimizer.zero_grad()
            (value_loss * args.value_loss_coef + action_loss - dist_entropy * args.entropy_coef).backward()

            if args.algo == 'a2c':
                nn.utils.clip_grad_norm(actor_critic.parameters(), args.max_grad_norm)

            optimizer.step()
        
        
        rollouts.after_update()

        if j % args.save_interval == 0 and args.save_dir != "":
            
            try:
                os.makedirs(save_path)
            except OSError:
                pass

            # A really ugly way to save a model to CPU
            save_model = actor_critic
            if args.cuda:
                save_model = copy.deepcopy(actor_critic).cpu()
                
            torch.save(save_model, SAVE_PATH)
            print("model saved to ", SAVE_PATH)

        if j % args.log_interval == 0:
            end = time.time()
            total_num_steps = (j + 1) * args.num_processes * args.num_steps
            print("Updates {}, num timesteps {}, FPS {}, mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}, entropy {:.5f}, value loss {:.5f}, policy loss {:.5f}".
                format(j, total_num_steps,
                       int(total_num_steps / (end - start)),
                       final_rewards.mean(),
                       final_rewards.median(),
                       final_rewards.min(),
                       final_rewards.max(), dist_entropy.data[0],
                       value_loss.data[0], action_loss.data[0]))
        if args.vis and j % args.vis_interval == 0:
            try:
                # Sometimes monitor doesn't properly flush the outputs
                win = visdom_plot(viz, win, args.log_dir, args.env_name, args.algo)
            except IOError:
                pass
            
main()

#######
#######
loading saved model from  saved_models/a2c/PongNoFrameskip-v4.pt
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 0, num timesteps 80, FPS 623, mean/median reward 0.0/0.0, min/max reward 0.0/0.0, entropy 1.78588, value loss 0.00337, policy loss -0.02857
Updates 100, num timesteps 8080, FPS 968, mean/median reward 0.0/0.0, min/max reward 0.0/0.0, entropy 1.76185, value loss 0.03304, policy loss 0.06543
Updates 200, num timesteps 16080, FPS 943, mean/median reward 0.0/0.0, min/max reward 0.0/0.0, entropy 1.66368, value loss 0.01249, policy loss -0.03710
Updates 300, num timesteps 24080, FPS 939, mean/median reward -1.0/0.0, min/max reward -16.0/0.0, entropy 1.76785, value loss 0.02095, policy loss 0.05166
Updates 400, num timesteps 32080, FPS 933, mean/median reward -1.0/0.0, min/max reward -16.0/0.0, entropy 1.76692, value loss 0.00879, policy loss 0.02968
Updates 500, num timesteps 40080, FPS 930, mean/median reward -5.0/0.0, min/max reward -16.0/0.0, entr

Updates 4900, num timesteps 392080, FPS 925, mean/median reward -4.3/-5.0, min/max reward -10.0/6.0, entropy 1.76280, value loss 0.00785, policy loss 0.01631
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 5000, num timesteps 400080, FPS 925, mean/median reward -4.2/-6.0, min/max reward -10.0/6.0, entropy 1.78170, value loss 0.03444, policy loss -0.03453
Updates 5100, num timesteps 408080, FPS 925, mean/median reward -3.8/-5.0, min/max reward -9.0/3.0, entropy 1.77313, value loss 0.05473, policy loss -0.01236
Updates 5200, num timesteps 416080, FPS 925, mean/median reward -5.2/-6.0, min/max reward -12.0/3.0, entropy 1.74454, value loss 0.09082, policy loss -0.12794
Updates 5300, num timesteps 424080, FPS 925, mean/median reward -6.0/-7.0, min/max reward -12.0/3.0, entropy 1.72363, value loss 0.01109, policy loss -0.05529
Updates 5400, num timesteps 432080, FPS 925, mean/median reward -6.2/-7.0, min/max reward -12.0/3.0, entropy 1.77720, value loss 0.02780, policy loss 0.

Updates 9900, num timesteps 792080, FPS 926, mean/median reward -4.9/-6.0, min/max reward -10.0/8.0, entropy 1.75985, value loss 0.07371, policy loss 0.13967
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 10000, num timesteps 800080, FPS 926, mean/median reward -3.8/-6.0, min/max reward -10.0/8.0, entropy 1.77162, value loss 0.01000, policy loss -0.00369
Updates 10100, num timesteps 808080, FPS 926, mean/median reward -3.2/-4.0, min/max reward -10.0/8.0, entropy 1.77689, value loss 0.03784, policy loss 0.01259
Updates 10200, num timesteps 816080, FPS 926, mean/median reward -3.2/-4.0, min/max reward -10.0/8.0, entropy 1.66208, value loss 0.07453, policy loss -0.02198
Updates 10300, num timesteps 824080, FPS 926, mean/median reward -3.6/-4.0, min/max reward -10.0/8.0, entropy 1.64997, value loss 0.07123, policy loss -0.07980
Updates 10400, num timesteps 832080, FPS 926, mean/median reward -3.8/-5.0, min/max reward -10.0/8.0, entropy 1.76504, value loss 0.03705, policy lo

Updates 14900, num timesteps 1192080, FPS 926, mean/median reward 3.4/4.0, min/max reward -3.0/12.0, entropy 1.76911, value loss 0.01287, policy loss 0.04571
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 15000, num timesteps 1200080, FPS 926, mean/median reward 2.9/4.0, min/max reward -3.0/12.0, entropy 1.78801, value loss 0.00867, policy loss 0.04264
Updates 15100, num timesteps 1208080, FPS 926, mean/median reward 4.4/5.0, min/max reward -3.0/12.0, entropy 1.74763, value loss 0.00829, policy loss 0.01315
Updates 15200, num timesteps 1216080, FPS 925, mean/median reward 4.8/5.0, min/max reward -2.0/12.0, entropy 1.77007, value loss 0.02806, policy loss -0.07889
Updates 15300, num timesteps 1224080, FPS 925, mean/median reward 5.2/5.0, min/max reward -2.0/12.0, entropy 1.74990, value loss 0.01536, policy loss -0.03107
Updates 15400, num timesteps 1232080, FPS 925, mean/median reward 5.0/5.0, min/max reward -2.0/12.0, entropy 1.77390, value loss 0.01966, policy loss 0.0

Updates 19900, num timesteps 1592080, FPS 920, mean/median reward 6.9/8.0, min/max reward -10.0/14.0, entropy 1.71690, value loss 0.02310, policy loss 0.00222
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 20000, num timesteps 1600080, FPS 920, mean/median reward 6.1/6.0, min/max reward -10.0/14.0, entropy 1.62491, value loss 0.16029, policy loss -0.08779
Updates 20100, num timesteps 1608080, FPS 920, mean/median reward 5.0/6.0, min/max reward -10.0/14.0, entropy 1.76196, value loss 0.01170, policy loss -0.00121
Updates 20200, num timesteps 1616080, FPS 920, mean/median reward 5.2/6.0, min/max reward -10.0/14.0, entropy 1.54789, value loss 0.02881, policy loss 0.02382
Updates 20300, num timesteps 1624080, FPS 919, mean/median reward 7.5/7.0, min/max reward 1.0/14.0, entropy 1.61150, value loss 0.10932, policy loss -0.11830
Updates 20400, num timesteps 1632080, FPS 919, mean/median reward 9.3/10.0, min/max reward 1.0/17.0, entropy 1.64219, value loss 0.03291, policy loss

Updates 24900, num timesteps 1992080, FPS 916, mean/median reward 13.6/14.0, min/max reward 3.0/19.0, entropy 1.72175, value loss 0.00424, policy loss 0.01807
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 25000, num timesteps 2000080, FPS 916, mean/median reward 14.0/14.0, min/max reward 3.0/19.0, entropy 1.70339, value loss 0.00753, policy loss -0.01848
Updates 25100, num timesteps 2008080, FPS 916, mean/median reward 14.0/14.0, min/max reward 3.0/19.0, entropy 1.77517, value loss 0.00867, policy loss -0.00310
Updates 25200, num timesteps 2016080, FPS 916, mean/median reward 14.3/15.0, min/max reward 3.0/19.0, entropy 1.74387, value loss 0.02335, policy loss 0.05952
Updates 25300, num timesteps 2024080, FPS 916, mean/median reward 14.1/14.0, min/max reward 7.0/18.0, entropy 1.73640, value loss 0.01612, policy loss -0.01772
Updates 25400, num timesteps 2032080, FPS 916, mean/median reward 14.3/15.0, min/max reward 7.0/18.0, entropy 1.62049, value loss 0.00741, policy l

Updates 29900, num timesteps 2392080, FPS 914, mean/median reward 9.9/9.0, min/max reward 3.0/16.0, entropy 1.59564, value loss 0.05919, policy loss -0.04358
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 30000, num timesteps 2400080, FPS 914, mean/median reward 9.4/9.0, min/max reward 3.0/16.0, entropy 1.63768, value loss 0.01695, policy loss 0.04703
Updates 30100, num timesteps 2408080, FPS 914, mean/median reward 10.3/10.0, min/max reward 3.0/16.0, entropy 1.69625, value loss 0.00704, policy loss 0.02764
Updates 30200, num timesteps 2416080, FPS 913, mean/median reward 11.1/12.0, min/max reward 3.0/16.0, entropy 1.73658, value loss 0.01586, policy loss 0.02589
Updates 30300, num timesteps 2424080, FPS 913, mean/median reward 10.2/10.0, min/max reward -2.0/16.0, entropy 1.70401, value loss 0.01936, policy loss 0.00575
Updates 30400, num timesteps 2432080, FPS 913, mean/median reward 10.8/12.0, min/max reward -2.0/16.0, entropy 1.66382, value loss 0.00532, policy loss 

Updates 34900, num timesteps 2792080, FPS 915, mean/median reward 12.9/12.0, min/max reward 6.0/20.0, entropy 1.76887, value loss 0.00831, policy loss -0.04415
model saved to  saved_models/a2c/PongNoFrameskip-v4.pt
Updates 35000, num timesteps 2800080, FPS 916, mean/median reward 13.4/12.0, min/max reward 8.0/20.0, entropy 1.66570, value loss 0.01261, policy loss 0.02815
Updates 35100, num timesteps 2808080, FPS 916, mean/median reward 13.0/12.0, min/max reward 7.0/20.0, entropy 1.72526, value loss 0.00701, policy loss 0.04530
Updates 35200, num timesteps 2816080, FPS 916, mean/median reward 13.6/14.0, min/max reward 7.0/18.0, entropy 1.70166, value loss 0.00564, policy loss 0.05194
Updates 35300, num timesteps 2824080, FPS 916, mean/median reward 12.6/12.0, min/max reward 7.0/18.0, entropy 1.49994, value loss 0.02040, policy loss 0.01479
Updates 35400, num timesteps 2832080, FPS 916, mean/median reward 12.7/12.0, min/max reward 7.0/18.0, entropy 1.67105, value loss 0.03660, policy los

NameError: name 'actor_critic' is not defined

True